## Test with pretrained models

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import datasets, transforms
import os
import time
import cpuinfo
from tqdm import tqdm

### File paths

In [2]:
train_path = "../data/Train"    # paths for your training and testing dataset
test_path = "../data/Test"
input_parameter = ""            # paths for import and export custom model trainable parameters
output_parameter = ""

### Pretrain Model of use from torchvision

In [3]:
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
print(f"Using model {type(model).__name__}")

epochs = 50
batch_size = 64 # adjust to your gpu vram
optimizer = optim.AdamW(
    model.parameters(),
    lr=3e-4,
    weight_decay=1e-4,
    betas=(0.9, 0.999),
)
loss_function = nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=epochs,
    eta_min=1e-6,
)

Using model ResNet


In [4]:
seed = 42
torch.manual_seed(seed)

### Data Preprocess Transformations

In [5]:
transform_train = transforms.Compose([  # on training dataset
    transforms.RandomRotation(degrees=15),  # introduce varieties for better generalization
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet1K stats
    ])

transform_test = transforms.Compose([   # on test dataset
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

### Data Loader

In [6]:
# Load datasets
train_dataset = datasets.ImageFolder(root=train_path, transform=transform_train)
test_dataset = datasets.ImageFolder(root=test_path, transform=transform_test)

print(f"CPU count: {os.cpu_count()}")
num_workers = min(4, os.cpu_count() // 2)  # Dynamically set num_workers
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print(f"Classes: {train_dataset.classes}")

CPU count: 16
Classes: ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']


In [7]:
# Edit the output layer of the model
num_classes = len(train_dataset.classes)
model.fc = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(in_features=model.fc.in_features, out_features=num_classes),
)

### Device of use

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

if device == torch.device("cuda"):
    print(f"Using GPU: {torch.cuda.get_device_name(device)}")
else:
    cpu_info = cpuinfo.get_cpu_info()
    print(f"Using CPU: {cpu_info['brand_raw']}")

Using GPU: NVIDIA GeForce RTX 3080


### Configure model parameters

In [9]:
# IF NEEDED
# Load custom weight and optimizer states
# if os.path.exists(input_parameter):
#     checkpoint = torch.load("test_weights.pth", map_location=device)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [10]:
# Selective layer freezing
# change base on your model
# "Early layers are often already well-optimized" by ChatGPT-O4
# for name, param in model.named_parameters():
#     if "fc" not in name:
#         param.requires_grad = False  # Freeze everything except fc

In [11]:
# Move model to device
model.to(device)
print(f"Model is on {next(model.parameters()).device}")

Model is on cuda:0


### Training Epochs

In [12]:
for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training loop with progress bar
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} [Train]", leave=False)
    for images, labels in train_bar:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_function(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate statistics
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Update progress bar
        train_bar.set_postfix({
            'loss': f"{running_loss / total_train:.4f}",
            'acc': f"{100. * correct_train / total_train:.2f}%"
        })

    # Calculate training statistics
    train_loss = running_loss / total_train
    train_acc = 100. * correct_train / total_train

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        val_bar = tqdm(test_loader, desc=f"Epoch {epoch + 1}/{epochs} [Val]", leave=False)
        for images, labels in val_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_function(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

            val_bar.set_postfix({
                'loss': f"{val_loss / total_val:.4f}",
                'acc': f"{100. * correct_val / total_val:.2f}%"
            })

    # Calculate validation statistics
    val_loss /= total_val
    val_acc = 100. * correct_val / total_val

    scheduler.step()

    # Print epoch summary
    print(f"\nEpoch {epoch + 1}/{epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


Epoch 1/50 - Train Loss: 1.6003, Train Acc: 44.66% | Val Loss: 1.8475, Val Acc: 41.53%



Epoch 2/50 - Train Loss: 1.2522, Train Acc: 55.34% | Val Loss: 1.5932, Val Acc: 47.46%



Epoch 3/50 - Train Loss: 1.0651, Train Acc: 62.44% | Val Loss: 1.5213, Val Acc: 50.00%



Epoch 4/50 - Train Loss: 1.0098, Train Acc: 64.49% | Val Loss: 1.4507, Val Acc: 53.39%



Epoch 5/50 - Train Loss: 0.9504, Train Acc: 67.31% | Val Loss: 1.4771, Val Acc: 56.78%


...


Epoch 46/50 - Train Loss: 0.2924, Train Acc: 88.61% | Val Loss: 1.6674, Val Acc: 61.86%



Epoch 47/50 - Train Loss: 0.2963, Train Acc: 88.30% | Val Loss: 1.6712, Val Acc: 61.86%



Epoch 48/50 - Train Loss: 0.2941, Train Acc: 88.16% | Val Loss: 1.6584, Val Acc: 63.56%



Epoch 49/50 - Train Loss: 0.2898, Train Acc: 88.74% | Val Loss: 1.6952, Val Acc: 61.86%



Epoch 50/50 - Train Loss: 0.2886, Train Acc: 88.74% | Val Loss: 1.6448, Val Acc: 61.86%


### Training log and data export

In [13]:
# Code here
# iteration 1
# Epoch 50/50 - Train Loss: 0.2886, Train Acc: 88.74% | Val Loss: 1.6448, Val Acc: 61.86%
# Time is 23m 17s